In [1]:
import argparse
import yaml
import pandas as pd

from tqdm import tqdm

In [2]:
with open("../config.yaml", 'r') as config_file:
    config = yaml.safe_load(config_file)

parser = argparse.ArgumentParser(description='') 
parser.set_defaults(**config)
args = parser.parse_args(args = [])

parser.add_argument("--prefix_path", type=str, default=f"../experiments/{args.date}_{args.experiment_name}_{args.mode}")
args = parser.parse_args(args = [])
args

Namespace(batch_size=32, date='0606', do_prediction=True, do_training=True, entity_type=['Company', 'Person'], experiment_name='bert_p=0.5_pos=sentence', lr='1e-5', max_len=512, mode='BP', n_epochs=5, num_labels=5, prefix_path='../experiments/0606_bert_p=0.5_pos=sentence_BP', pretrained_model='hfl/chinese-roberta-wwm-ext', test_data='/test_dataset', threshold=0.5, train_data='/train_dataset', valid_data='/valid_dataset')

In [3]:
df_test_path = args.prefix_path + "/test_data.pkl"
df_output_path = args.prefix_path + "/test_data_predict_result.pkl"
df_save_path = args.prefix_path + "/test_data_predict_result_all.xlsx"

In [4]:
# df_test_path = args.prefix_path + "/badcase_df.pkl"
# df_output_path = args.prefix_path + "/test_data_predict_result.pkl"
# df_save_path = args.prefix_path + "/badcase_data_predict_result_all.xlsx"

In [5]:
df_test = pd.read_pickle(df_test_path)
df_test.shape

(4554, 20)

In [6]:
df_test.head()

,docid,headline,content,ner,matched_keywords,include_company,include_person,length,split_sentence_index,Companys,Persons,context_cleaned,person_matched,company_matched,context_keywords,sign,input_text,tokenized_content,ids,masks
0,2023063021774796605&&2,NaN,"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...","[{'label_name': 'Company', 'text_segment': '21...",['黄如'],True,True,471,2,[],['黄如'],"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...",[黄如],[],"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...",False,"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...","[突, 出, 科, 产, 融, 合, ,, 推, 动, 创, 新, 链, 、, 产, 业, ...","[101, 4960, 1139, 4906, 772, 6084, 1394, 117, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,2023063021774797155&&0,NaN,曾国卫受邀回母校 鼓励同学深入认识祖国。政制及内地事务局局长曾国卫今日(30日)受邀回到母校...,"[{'label_name': 'Person', 'text_segment': '曾國衞...","['长和', '曾国卫', '赵霁']",True,True,496,0,['长和'],"['曾国卫', '赵霁']",曾国卫受邀回母校鼓励同学深入认识祖国。政制及内地事务局局长曾国卫今日(30日)受邀回到母校东...,"[曾国卫, 赵霁]",[长和],曾国卫受邀回母校鼓励同学深入认识祖国。||可能的实体：曾国卫||政制及内地事务局局长曾国卫今...,False,曾国卫受邀回母校鼓励同学深入认识祖国。政制及内地事务局局长曾国卫今日(30日)受邀回到母校东...,"[曾, 国, 卫, 受, 邀, 回, 母, 校, 鼓, 励, 同, 学, 深, 入, 认, ...","[101, 3295, 1744, 1310, 1358, 6913, 1726, 3678...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2023063021774797862&&1,NaN,初步上市申请文件披露，黑芝麻2016年成立，上市前已进行10轮融资，入股价由A轮的每股0.1...,"[{'label_name': 'Company', 'text_segment': '黑芝...","['汽车', '小米', 'SoC', '中国银行', '腾讯', '黑芝麻集资', 'Oc...",True,True,482,1,"['小米', '黑芝', 'Oceanpine', '北极光创投', '汽车', '黑芝麻'...",[],初步上市申请文件披露，黑芝麻2016年成立，上市前已进行10轮融资，入股价由a轮的每股0.1...,[],"[汽车, 小米, SoC, 中国银行, 腾讯, 黑芝麻集资, Oceanpine, 吉利集团...",初步上市申请文件披露，黑芝麻2016年成立，上市前已进行10轮融资，入股价由a轮的每股0.1...,False,初步上市申请文件披露，黑芝麻2016年成立，上市前已进行10轮融资，入股价由a轮的每股0.1...,"[初, 步, 上, 市, 申, 请, 文, 件, 披, 露, ，, 黑, 芝, 麻, 2, ...","[101, 1159, 3635, 677, 2356, 4509, 6435, 3152,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,2023063021774798050&&0,NaN,新明中国(02699) 内幕消息 / 停牌。新明中国(02699) 复牌进度的季度更新(30...,"[{'label_name': 'Company', 'text_segment': '新明...",['新明中国'],True,True,55,0,['新明中国'],[],新明中国(02699) 内幕消息 / 停牌。新明中国(02699) 复牌进度的季度更新(30...,[],[新明中国],新明中国(02699) 内幕消息 / 停牌。||可能的实体：新明中国||新明中国(02699...,False,新明中国(02699) 内幕消息 / 停牌。新明中国(02699) 复牌进度的季度更新(30...,"[新, 明, 中, 国, (, 0, 2, 6, 9, 9, ), 内, 幕, 消, 息, ...","[101, 3173, 3209, 704, 1744, 113, 121, 123, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,2023063021774805645&&0,NaN,域多利道地皮接6标书。地政总署公布，坚尼地城西宁街与域多利道交界卖地表地皮(下称域多利道地皮...,"[{'label_name': 'Company', 'text_segment': '地政...","['新地', '长实', '会德丰地产', '信置', '嘉华国际', '交通', '招商局...",True,True,293,0,"['新地', '长实', '会德丰地产', '信置', '嘉华国际', '交通', '招商局...",[],域多利道地皮接6标书。地政总署公布，坚尼地城西宁街与域多利道交界卖地表地皮(下称域多利道地皮...,[],"[新地, 长实, 会德丰地产, 信置, 嘉华国际, 交通, 招商局置地, 城巴, 鹰君, 东华]",域多利道地皮接6标书。地政总署公布，坚尼地城西宁街与域多利道交界卖地表地皮(下称域多利道地皮...,False,域多利道地皮接6标书。地政总署公布，坚尼地城西宁街与域多利道交界卖地表地皮(下称域多利道地皮...,"[域, 多, 利, 道, 地, 皮, 接, 6, 标, 书, 。, 地, 政, 总, 署, ...","[101, 1818, 1914, 1164, 6887, 1765, 4649, 2970...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [7]:
df_output = pd.read_pickle(df_output_path)
df_output.shape

(4554, 5)

In [8]:
df_all = pd.merge(df_test, df_output, on='docid', how='left')
df_all['input_len'] = df_all["input_text"].str.len()
df_all.shape

(4554, 25)

In [9]:
df_all.head()

,docid,headline,content,ner,matched_keywords,include_company,include_person,length,split_sentence_index,Companys,...,sign,input_text,tokenized_content_x,ids,masks,tokenized_content_y,label_pred,Company_pred,Person_pred,input_len
0,2023063021774796605&&2,NaN,"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...","[{'label_name': 'Company', 'text_segment': '21...",['黄如'],True,True,471,2,[],...,False,"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...","[突, 出, 科, 产, 融, 合, ,, 推, 动, 创, 新, 链, 、, 产, 业, ...","[101, 4960, 1139, 4906, 772, 6084, 1394, 117, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[突, 出, 科, 产, 融, 合, ,, 推, 动, 创, 新, 链, 、, 产, 业, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],[黄如],471
1,2023063021774797155&&0,NaN,曾国卫受邀回母校 鼓励同学深入认识祖国。政制及内地事务局局长曾国卫今日(30日)受邀回到母校...,"[{'label_name': 'Person', 'text_segment': '曾國衞...","['长和', '曾国卫', '赵霁']",True,True,496,0,['长和'],...,False,曾国卫受邀回母校鼓励同学深入认识祖国。政制及内地事务局局长曾国卫今日(30日)受邀回到母校东...,"[曾, 国, 卫, 受, 邀, 回, 母, 校, 鼓, 励, 同, 学, 深, 入, 认, ...","[101, 3295, 1744, 1310, 1358, 6913, 1726, 3678...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[曾, 国, 卫, 受, 邀, 回, 母, 校, 鼓, 励, 同, 学, 深, 入, 认, ...","[B-Person, I-Person, I-Person, O, O, O, O, O, ...","[steamroom, 东莞同乡会方树泉学校]","[赵霁, 曾国卫]",495
2,2023063021774797862&&1,NaN,初步上市申请文件披露，黑芝麻2016年成立，上市前已进行10轮融资，入股价由A轮的每股0.1...,"[{'label_name': 'Company', 'text_segment': '黑芝...","['汽车', '小米', 'SoC', '中国银行', '腾讯', '黑芝麻集资', 'Oc...",True,True,482,1,"['小米', '黑芝', 'Oceanpine', '北极光创投', '汽车', '黑芝麻'...",...,False,初步上市申请文件披露，黑芝麻2016年成立，上市前已进行10轮融资，入股价由a轮的每股0.1...,"[初, 步, 上, 市, 申, 请, 文, 件, 披, 露, ，, 黑, 芝, 麻, 2, ...","[101, 1159, 3635, 677, 2356, 4509, 6435, 3152,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[初, 步, 上, 市, 申, 请, 文, 件, 披, 露, ，, 黑, 芝, 麻, 2, ...","[O, O, O, O, O, O, O, O, O, O, O, B-Company, I...","[黑芝麻, 中国银行, 北极光创投, 小米, 汽车, 吉利集团, 腾讯]",[],482
3,2023063021774798050&&0,NaN,新明中国(02699) 内幕消息 / 停牌。新明中国(02699) 复牌进度的季度更新(30...,"[{'label_name': 'Company', 'text_segment': '新明...",['新明中国'],True,True,55,0,['新明中国'],...,False,新明中国(02699) 内幕消息 / 停牌。新明中国(02699) 复牌进度的季度更新(30...,"[新, 明, 中, 国, (, 0, 2, 6, 9, 9, ), 内, 幕, 消, 息, ...","[101, 3173, 3209, 704, 1744, 113, 121, 123, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[新, 明, 中, 国, (, 0, 2, 6, 9, 9, ), 内, 幕, 消, 息, ...","[B-Company, I-Company, I-Company, I-Company, O...",[新明中国],[],55
4,2023063021774805645&&0,NaN,域多利道地皮接6标书。地政总署公布，坚尼地城西宁街与域多利道交界卖地表地皮(下称域多利道地皮...,"[{'label_name': 'Company', 'text_segment': '地政...","['新地', '长实', '会德丰地产', '信置', '嘉华国际', '交通', '招商局...",True,True,293,0,"['新地', '长实', '会德丰地产', '信置', '嘉华国际', '交通', '招商局...",...,False,域多利道地皮接6标书。地政总署公布，坚尼地城西宁街与域多利道交界卖地表地皮(下称域多利道地皮...,"[域, 多, 利, 道, 地, 皮, 接, 6, 标, 书, 。, 地, 政, 总, 署, ...","[101, 1818, 1914, 1164, 6887, 1765, 4649, 2970...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[域, 多, 利, 道, 地, 皮, 接, 6, 标, 书, 。, 地, 政, 总, 署, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[交通, 鹰君, 新地, 长实, 巴士, 会德丰地产, 嘉华国际, 信置, 招商局置地]",[],293


In [10]:
df_all.to_excel(df_save_path, index=False, engine='xlsxwriter')

In [11]:
df_all.columns

Index(['docid', 'headline', 'content', 'ner', 'matched_keywords',
       'include_company', 'include_person', 'length', 'split_sentence_index',
       'Companys', 'Persons', 'context_cleaned', 'person_matched',
       'company_matched', 'context_keywords', 'sign', 'input_text',
       'tokenized_content_x', 'ids', 'masks', 'tokenized_content_y',
       'label_pred', 'Company_pred', 'Person_pred', 'input_len'],
      dtype='object')

In [12]:
from zhconv import convert
df_all['content'] = df_all['content'].apply(lambda content: convert(content, 'zh-cn'))
df_all.sample(200, random_state=2)[['docid', 'content', "person_matched", "company_matched", 'Company_pred', 'Person_pred']].to_excel("/workspace/NER/testset_sample.xlsx", index=False)

## eval

In [13]:
from evalner import partial_match

In [14]:
# df_all = df_all[df_all['input_len'] <= 510]

df_all.reset_index(drop=True, inplace=True)

In [15]:
df_all.shape

(4554, 25)

In [16]:
def findBoundary(val, text):
    # 寻找抽取到的实体的边界索引
    res = []
    for i in range(0, len(text) - len(val) + 1):
        if text[i:i + len(val)] == val:
            res.append((i, i + len(val)))
    return res

def checkIfOverlap(pred_val, true_val, text):
    # 检查两个实体是否部分匹配，该方法中intersec是判断两个实体的索引是否有交集，有交集即为True, 还有严格的计算标准就是是否完全匹配
    rang_a = findBoundary(true_val, text)
    rang_b = findBoundary(pred_val, text)
    # print(rang_a, rang_b)
    if len(rang_a) == 0 or len(rang_b) == 0:
        return False
    else:
        for i, j in rang_a:
            for k, m in rang_b:
                if (i, j) == (k, m):
                    return True
                else:
                    return False
                # 计算两个实体的索引是否有交集，如果有交集即是实体部分匹配
                # intersec = set(range(i, j)).intersection(set(range(k, m)))
                # if len(intersec) > 0:
                #     return True
                # else:
                #     return False
                
def partial_match(pred_list, golden_list, text):
    precision_hit, precision_all = 0, 0
    recall_hit, recall_all = 0, 0
    
    for pred in pred_list:
        precision_all += 1
        for golden in golden_list:
            if checkIfOverlap(pred, golden, text):
                precision_hit += 1
                break
    
    for golden in golden_list:
        recall_all += 1
        for pred in pred_list:
            if checkIfOverlap(pred, golden, text):
                recall_hit += 1
                break

    return precision_hit, precision_all, recall_hit, recall_all


In [17]:
result = {}
for entity_type in tqdm(args.entity_type):
    p_hit, p_all, r_hit, r_all = 0, 0, 0, 0
    for _, row in df_all.iterrows():
        # print(row)
        golden_list = row[f'{entity_type}_matched'.lower()] ##
        pred_list = row[f'{entity_type}_pred'] ##
        # text = str(row['context_cleaned']) ##
        text = str(row['input_text']) ##
        # print(golden_list, pred_list, text)
        p_hit_, p_all_, r_hit_, r_all_ = partial_match(pred_list, golden_list, text)
        p_hit += p_hit_
        p_all += p_all_
        r_hit += r_hit_
        r_all += r_all_
    
    p = p_hit / p_all
    r = r_hit / r_all
    
    result.update(
        {
            f"{entity_type}": {
                "precision": round(p, 4),
                "recall": round(r, 4),
                "f1": round(2 * p * r / (p + r), 4),
                "details": [p_hit, p_all, r_hit, r_all]
            }
        }
    )


100%|██████████| 2/2 [00:16<00:00,  8.27s/it]


In [18]:
result

{'Company': {'precision': 0.796,
  'recall': 0.7355,
  'f1': 0.7645,
  'details': [10267, 12899, 10267, 13960]},
 'Person': {'precision': 0.8643,
  'recall': 0.8446,
  'f1': 0.8543,
  'details': [4912, 5683, 4912, 5816]}}

## precision

In [19]:
!pip install fuzzywuzzy==0.18.0


[notice] A new release of pip is available: 23.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [20]:
from muc import evaluate_all

def run_eval4ner(df, entity_type):
    # docid_remove = [df_row['docid'] for _, df_row in df.iterrows() if (df_row[f'{entity_type}_pred'] == []) or (df_row["input_len"] > 510)] #  and (df_row[f'{entity_type}_pred'] == [])(df_row[f'{entity_type}_gt'] == [])
    # df_eval = df[~df['docid'].isin(docid_remove)]
    df_eval = df
    print("Number of data for evaluation:", len(df_eval))

    pred = [[(f"{entity_type}", j.lower()) for j in i] for i in df_eval[f'{entity_type}_pred'].to_list()]
    label_gt = [[(f"{entity_type}", j.lower()) for j in i] for i in df_eval[f'{entity_type}_matched'.lower()].to_list()]
     
    print(f"======= {entity_type} =======\nExample:\ncontent:{df_eval['content'].tolist()[0]},\npred: {pred[0]}\nlabel: {label_gt[0]}")
    evaluate_all(pred, label_gt, texts=df_eval['input_text'].to_list())

    print("\n\n")

In [21]:
for entity_type in args.entity_type:
    run_eval4ner(df_all, entity_type)

Number of data for evaluation: 4554
======= Company =======
Example:
content:突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题,聚力产出一批高水平标志性科技成果,更好带动产业集聚集群发展。突出行业融合,积极探索新型会员模式,开展共性技术难题攻关,强化行业内部创新资源融合共享,形成多方共建共治共享的治理结构。希望国创中心加快实体化运作,在理事会领导下高效执行、快速起跑,全力推动各项工作有序开展,真正把国创中心建设成为集成电路设计自动化产业的技术发源地、产品示范地和人才汇聚地。东南大学校长黄如表示,要努力将EDA国创中心打造成国内领先、世界一流的EDA技术创新中心、人才培养基地和产业孵化重镇。以揭牌为起点,未来中心将提高站位,坚持"全国创新中心"核心定位,充分整合国内外优质资源,构建高校领衔、企业加盟、产学研深度融合的新型科技攻关体制,打通EDA领域上下游合作机制,形成全国一盘棋、全球一张网的科技研发力量,合力打造EDA"国家队"。围绕国家战略和产业需求,要坚持近期目标与长远目标相结合、当前技术与下一代技术相结合的原则,科学制定工作规划和项目目标,尽快落地标志性成果,
pred: []
label: []

 NER evaluation scores:
  strict mode, Precision=0.7769, Recall=0.7468, F1:0.7430
   exact mode, Precision=0.7769, Recall=0.7468, F1:0.7430
 partial mode, Precision=0.8493, Recall=0.8062, F1:0.8075
    type mode, Precision=0.8493, Recall=0.8062, F1:0.8075
 similar mode, Precision=0.8515, Recall=0.8079, F1:0.8096



Number of data for evaluation: 4554
======= Person =======
Example:
content:突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础

## recall

In [22]:
from muc import evaluate_all

def run_eval4ner(df, entity_type):
    docid_remove_gt = [df_row['docid'] for _, df_row in df.iterrows() if (df_row[f'{entity_type}_matched'.lower()] == []) or (df_row["input_len"] > 510)] #  and (df_row[f'{entity_type}_pred'] == [])(df_row[f'{entity_type}_gt'] == [])
  
    df_eval_gt = df[~df['docid'].isin(docid_remove_gt)]
 
    print("Number of data for evaluation:", len(df_eval_gt))
 
    pred_gpt = [[(f"{entity_type}", j.lower()) for j in i] for i in df_eval_gt[f'{entity_type}_pred'].to_list()]
    label_gpt = [[(f"{entity_type}", j.lower()) for j in i] for i in df_eval_gt[f'{entity_type}_matched'.lower()].to_list()]
   
    print(f"======= {entity_type} =======\nExample:\npred: {pred_gpt[0]}\nGPT: {label_gpt[0]}")
    evaluate_all(pred_gpt, label_gpt, texts=df_eval_gt['input_text'].to_list())

    print("\n\n")

In [23]:
for entity_type in args.entity_type:
    run_eval4ner(df_all, entity_type)

Number of data for evaluation: 4132
======= Company =======
Example:
pred: [('Company', 'steamroom'), ('Company', '东莞同乡会方树泉学校')]
GPT: [('Company', '长和')]

 NER evaluation scores:
  strict mode, Precision=0.7734, Recall=0.7406, F1:0.7363
   exact mode, Precision=0.7734, Recall=0.7406, F1:0.7363
 partial mode, Precision=0.8528, Recall=0.8057, F1:0.8071
    type mode, Precision=0.8528, Recall=0.8057, F1:0.8071
 similar mode, Precision=0.8553, Recall=0.8077, F1:0.8094



Number of data for evaluation: 2469
======= Person =======
Example:
pred: [('Person', '黄如')]
GPT: [('Person', '黄如')]

 NER evaluation scores:
  strict mode, Precision=0.8391, Recall=0.8382, F1:0.8277
   exact mode, Precision=0.8391, Recall=0.8382, F1:0.8277
 partial mode, Precision=0.8575, Recall=0.8544, F1:0.8448
    type mode, Precision=0.8575, Recall=0.8544, F1:0.8448
 similar mode, Precision=0.8644, Recall=0.8611, F1:0.8515





In [24]:
def compute_f1(p, r):
    return 2*p*r / (p + r)

compute_f1(0.9599, 0.9622)

0.9610486239009417

# 对badcase数据集进行当前模型的测试

In [25]:
badcase_data = pd.read_excel("/workspace/client_project/poc/bochk/badcase/bochk_uat_20240410_ner.xlsx")

In [26]:
badcase_data.head(2).to_dict("records")

[{'doc_id': 2023120100000939360,
  'headline': '英再有城市破產 諾定咸超支2.3億',
  'content': '\n繼英國第二大城市伯明翰市議會9月初宣布破產後，英國中部城市諾定咸（Nottingham）市議會也於周三（29日）宣布破產，當局歸因於福利需求上升及通脹等全國性因素。\n\n事實上，市議會在理財上一直為人詬病，\n\n...\n\n訂閱hket.com即送 GNC男士綜合維他命加強版 (價值$268)\n\n詳情\n\n登入了解詳情\n',
  'ner_label_type': 'Company',
  'ner_keyword': 'Nottingham'},
 {'doc_id': 2023120100001090313,
  'headline': '中央谷長三角 寧滬盈利復增',
  'content': '  江蘇寧滬高速（00177）主業務是收費路橋的投資、建設、營運和管理，經營區域在長江三角洲，是江蘇省唯一上市路橋公司。而公司盈利經過2022年倒退後，今年可以恢復增長，中央重申推動長江經濟帶高質量發展，   訂閱hket.com即送 GNC男士綜合維他命加強版 (價值$268) 詳情 \n\n 登入 \n\n 了解詳情 \n\n 撰文 : 余少文 \n\n 欄名 : 中字股實力談\n',
  'ner_label_type': 'Company',
  'ner_keyword': '江蘇寧滬高速'}]

In [27]:
!cp /workspace/NER/experiments/0606_bert_p\=0.5_pos\=sentence_BP/model.pth /workspace/ner-starbucks/src/models/model_0606.pth

# 使用新接口对测试集进行ner打标并统计结果

In [28]:
df_test.shape

(4554, 20)

In [29]:
from tqdm import tqdm
import requests
df_test_list = df_test.to_dict('record')
# url = "http://ess76.wisers.com:13352/ner/content"

url = "http://ess73.wisers.com:13347/ner/content"
ner_tag_result = []
for row in tqdm(df_test_list):
    input_param = {
        "docid": row['docid'],
        "content": row["content"],
        "add_keywords": False
    }
    response = requests.post(url, json=input_param)
    try:
        if response.status_code == 200:
            result = response.json()
            ner_result = result['retData']
            dict_matched_company = []
            dict_matched_person = []
            algorithm_matched_company = []
            algorithm_matched_person = []

            for info in ner_result['text']:
                if info['label_name'] == 'Company':
                    if info['resource'] == 'Algorithm prediction':
                        algorithm_matched_company.append(info['text_segment'])
                    elif info['resource'] == "Dictionary matching":
                        dict_matched_company.append(info['text_segment'])
                elif info['label_name'] == 'Person':
                    if info['resource'] == 'Algorithm prediction':
                        algorithm_matched_person.append(info['text_segment'])
                    elif info['resource'] == "Dictionary matching":
                        dict_matched_person.append(info['text_segment'])
                        
            ner_tag_result.append({
                "docid": row['docid'],
                "ner_result": ner_result,
                "dict_matched_company": dict_matched_company,
                "dict_matched_person": dict_matched_person,
                "algorithm_matched_company": algorithm_matched_company,
                "algorithm_matched_person": algorithm_matched_person,
            })
    except Exception as e:
        print(e, response)

                        
    
        

/tmp/ipykernel_62410/1095725994.py:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  df_test_list = df_test.to_dict('record')
100%|██████████| 4554/4554 [04:04<00:00, 18.60it/s]


In [30]:
ner_tag_add_origin_data = df_test.merge(pd.DataFrame(ner_tag_result), on='docid', how='left')

In [31]:
ner_tag_add_origin_data.columns

Index(['docid', 'headline', 'content', 'ner', 'matched_keywords',
       'include_company', 'include_person', 'length', 'split_sentence_index',
       'Companys', 'Persons', 'context_cleaned', 'person_matched',
       'company_matched', 'context_keywords', 'sign', 'input_text',
       'tokenized_content', 'ids', 'masks', 'ner_result',
       'dict_matched_company', 'dict_matched_person',
       'algorithm_matched_company', 'algorithm_matched_person'],
      dtype='object')

In [32]:
ner_tag_add_origin_data[['docid', 'content', 'ner', 'ner_result', 'person_matched', 'company_matched', 'dict_matched_company', 'dict_matched_person',
       'algorithm_matched_company', 'algorithm_matched_person']].to_excel("test_ner_api_result.xlsx", index=False)

In [33]:
ner_tag_add_origin_data["Company_pred"] = ner_tag_add_origin_data['algorithm_matched_company']
ner_tag_add_origin_data["Person_pred"] = ner_tag_add_origin_data['algorithm_matched_person']
ner_tag_add_origin_data['input_len'] = ner_tag_add_origin_data['content'].apply(len)
for entity_type in args.entity_type:
    run_eval4ner(ner_tag_add_origin_data, entity_type)

Number of data for evaluation: 4132
======= Company =======
Example:
pred: [('Company', '东莞同乡会方树泉学校'), ('Company', '东莞同乡会方树泉学校'), ('Company', '东莞同乡会方树泉学校'), ('Company', 'steam'), ('Company', '东莞同乡会方树泉学校')]
GPT: [('Company', '长和')]

 NER evaluation scores:
  strict mode, Precision=0.7480, Recall=0.7158, F1:0.7104
   exact mode, Precision=0.7480, Recall=0.7158, F1:0.7104
 partial mode, Precision=0.8366, Recall=0.7844, F1:0.7875
    type mode, Precision=0.8366, Recall=0.7844, F1:0.7875
 similar mode, Precision=0.8378, Recall=0.7849, F1:0.7884



Number of data for evaluation: 2469
======= Person =======
Example:
pred: [('Person', '黄如')]
GPT: [('Person', '黄如')]

 NER evaluation scores:
  strict mode, Precision=0.8051, Recall=0.8057, F1:0.7933
   exact mode, Precision=0.8051, Recall=0.8057, F1:0.7933
 partial mode, Precision=0.8324, Recall=0.8288, F1:0.8182
    type mode, Precision=0.8324, Recall=0.8288, F1:0.8182
 similar mode, Precision=0.8327, Recall=0.8289, F1:0.8185





In [34]:
ner_tag_add_origin_data["Company_pred"] = ner_tag_add_origin_data.apply(lambda row: row['algorithm_matched_company'] + row['dict_matched_company'], axis=1)
ner_tag_add_origin_data["Person_pred"] = ner_tag_add_origin_data.apply(lambda row: row['algorithm_matched_person'] + row['dict_matched_person'], axis=1)

for entity_type in args.entity_type:
    run_eval4ner(ner_tag_add_origin_data, entity_type)

Number of data for evaluation: 4132
======= Company =======
Example:
pred: [('Company', '东莞同乡会方树泉学校'), ('Company', '东莞同乡会方树泉学校'), ('Company', '东莞同乡会方树泉学校'), ('Company', 'steam'), ('Company', '东莞同乡会方树泉学校'), ('Company', '政制及内地事务局'), ('Company', '政制及内地事务局')]
GPT: [('Company', '长和')]

 NER evaluation scores:
  strict mode, Precision=0.7436, Recall=0.7775, F1:0.7379
   exact mode, Precision=0.7436, Recall=0.7775, F1:0.7379
 partial mode, Precision=0.8521, Recall=0.8370, F1:0.8237
    type mode, Precision=0.8521, Recall=0.8370, F1:0.8237
 similar mode, Precision=0.8535, Recall=0.8375, F1:0.8248



Number of data for evaluation: 2469
======= Person =======
Example:
pred: [('Person', '黄如')]
GPT: [('Person', '黄如')]

 NER evaluation scores:
  strict mode, Precision=0.8051, Recall=0.8057, F1:0.7933
   exact mode, Precision=0.8051, Recall=0.8057, F1:0.7933
 partial mode, Precision=0.8324, Recall=0.8288, F1:0.8182
    type mode, Precision=0.8324, Recall=0.8288, F1:0.8182
 similar mode, Precision=0.

In [35]:
ner_tag_add_origin_data["Company_pred"] = ner_tag_add_origin_data['dict_matched_company']
ner_tag_add_origin_data["Person_pred"] = ner_tag_add_origin_data['dict_matched_person']
for entity_type in args.entity_type:
    run_eval4ner(ner_tag_add_origin_data, entity_type)

Number of data for evaluation: 4132
======= Company =======
Example:
pred: [('Company', '政制及内地事务局'), ('Company', '政制及内地事务局')]
GPT: [('Company', '长和')]

 NER evaluation scores:
  strict mode, Precision=0.1276, Recall=0.0617, F1:0.0745
   exact mode, Precision=0.1276, Recall=0.0617, F1:0.0745
 partial mode, Precision=0.2350, Recall=0.1082, F1:0.1350
    type mode, Precision=0.2350, Recall=0.1082, F1:0.1350
 similar mode, Precision=0.2358, Recall=0.1084, F1:0.1354



Number of data for evaluation: 2469
======= Person =======
Example:
pred: []
GPT: [('Person', '黄如')]

 NER evaluation scores:
  strict mode, Precision=0.0000, Recall=0.0000, F1:0.0000
   exact mode, Precision=0.0000, Recall=0.0000, F1:0.0000
 partial mode, Precision=0.0000, Recall=0.0000, F1:0.0000
    type mode, Precision=0.0000, Recall=0.0000, F1:0.0000
 similar mode, Precision=0.0000, Recall=0.0000, F1:0.0000





2000.0